# Sep 28th, 2021

**Motivation**: 0.1 and 0.2 <br>

In [1]:
# HIDE CODE


import os
import sys
import nibabel as nib
import networkx as nx
from time import time
from pprint import pprint
from copy import deepcopy as dc
from os.path import join as pjoin
from myterial import orange, blue_grey
from IPython.display import display, IFrame, HTML
from matplotlib.colors import rgb2hex, to_rgb
import matplotlib.pyplot as plt
import seaborn as sns

# set style & no interpolalation
import matplotlib
matplotlib.rcParams['image.interpolation'] = 'none'
sns.set_style('whitegrid')

# tmp & extras dir
git_dir = pjoin(os.environ['HOME'], 'Dropbox/git')
tmp_dir = pjoin(git_dir, 'jb-Ca-fMRI/tmp')
extras_dir = pjoin(git_dir, 'jb-Ca-fMRI/_extras')
lfr_dir = pjoin(os.environ['HOME'], 'Documents/workspaces/lfr/binary_overlapping')


# GitHub
sys.path.insert(0, pjoin(git_dir, '_Ca-fMRI'))
from register.atlas import load_allen, make_tree_graph
from register.parcellation import Parcellation
from analysis.hierarchical import Hierarchical
from analysis.bootstrap import *
from analysis.svinet import *
from analysis.group import *
from analysis.lfr import *
from utils.render import *
from utils.plotting import *
from model.mouse import Mice
from model.configuration import Config

# warnings
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [2]:
mice = Mice(Config(32), load_preproc=True)

In [3]:
from register.register import *

In [ ]:
for resolution in [100, 200]:
    reg = Register(mice, resolution=resolution)

    df_tx, best_tx, perf = reg.fit_register(
        mode='3d',
        name=f'yale-{resolution/1000}_2_nissl',
        fixed=reg.nissl,
        moving=reg.in_house,
        save_results=False,
        types=['SyN', 'ElasticSyN', 'SyNAggro'],
        grads=[0.2, 0.25, 0.3],
        syn_bins=[16, 32],
        aff_bins=[13, 20, 32],
        aff_rates=[0.2, 0.8],
        n_seeds=10,
        mi_q=0.75,
    )
    df_tx.drop('tx', axis=1).to_pickle(f"./sep28/yale-{resolution/1000}.df")
    iso_yale = ants.apply_transforms(
        fixed=reg.in_house,
        moving=reg.objects['ISO'],
        transformlist=best_tx['invtransforms'],
        interpolator='genericLabel',
    )
    vis_yale = ants.apply_transforms(
        fixed=reg.in_house,
        moving=reg.objects['VIS'],
        transformlist=best_tx['invtransforms'],
        interpolator='genericLabel',
    )
    ants.image_write(iso_yale, f"./sep28/iso_yale-{resolution/1000}_Best.nii.gz")
    ants.image_write(vis_yale, f"./sep28/vis_yale-{resolution/1000}_Best.nii.gz")
    ants.image_write(reg.in_house, f"./sep28/yale-{resolution/1000}.nii.gz")
    
    for axis in range(3):
        reg.in_house.plot(
            overlay=iso_yale,
            overlay_cmap='Reds',
            overlay_alpha=0.5,
            axis=axis,
            nslices=49,
            filename=f"./sep28/yale-{resolution/1000}_Best_axis{axis}.png",
        )

  0%|          | 0/10 [00:00<?, ?it/s]